### Topics

- Tidying HTML
- simple parallelization
- Storing Snapshots by using python dill
- Exploration
- Stock selection using simple filters

### Imports

In [1]:
import numpy as np # linear algebra
import pandas as pd # pandas for dataframe based data processing and CSV file I/O

In [6]:
import requests # for http requests
from bs4 import BeautifulSoup # for html parsing and scraping
#!pip install fastnumbers
from fastnumbers import isfloat 
from fastnumbers import fast_float
from multiprocessing.dummy import Pool as ThreadPool 
import bs4
import matplotlib.pyplot as plt
import seaborn as sns
import json
!pip install pytidylib
from tidylib import tidy_document # for tidying incorrect html
from IPython.core.display import HTML

sns.set_style('whitegrid')
%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"



  Created wheel for pytidylib: filename=pytidylib-0.3.2-py3-none-any.whl size=8551 sha256=452c6430d5256853c4af1ac06777c96243dc5455b9bdf9acd28fd6622a463f2e
  Stored in directory: c:\users\ainap\appdata\local\pip\cache\wheels\9b\e4\23\8f81a76c9f2535ee091f118948202c8821ac671b7dbe41208b
Successfully built pytidylib


In [7]:
def ffloat(string):
    if string is None:
        return np.nan
    if type(string)==float or type(string)==np.float64:
        return string
    if type(string)==int or type(string)==np.int64:
        return string
    return fast_float(string.split(" ")[0].replace(',','').replace('%',''),
                      default=np.nan)

def ffloat_list(string_list):
    return list(map(ffloat,string_list))

def remove_multiple_spaces(string):
    if type(string)==str:
        return ' '.join(string.split())
    return string

def get_children(html_content):
    children = list()
    for item in html_content.children:
        if type(item)==bs4.element.Comment:
            continue
        if type(item)==bs4.element.Tag or len(str(item).replace("\n","").strip())>0:
            children.append(item)
        
    return children

In [8]:


def get_table_simple(table,is_table_tag=True):
    elems = table.find_all('tr') if is_table_tag else get_children(table)
    table_data = list()
    for row in elems:
        
        row_data = list()
        row_elems = get_children(row)
        for elem in row_elems:
            text = elem.text.strip().replace("\n","")
            text = remove_multiple_spaces(text)
            if len(text)==0:
                continue
            row_data.append(text)
        table_data.append(row_data)
    return table_data

        

### HTML Tidying

#### Without Tidying

We remove a closing </tr> from previous html and see if BeautifulSoup can parse it correctly. Look for closing tr of July

In [9]:
html = '''
<table>
    <tr>
        <td>Month</td>
        <td>Price</td>
    </tr>
    <tr>
        <td>July</td>
        <td>2</td>
    
    <tr>
        <td>August</td>
        <td>4</td>
    </tr>
    <tr>
        <td>September</td>
        <td>3</td>
    </tr>
    <tr>
        <td>October</td>
        <td>2</td>
    </tr>
</table>

'''

content = BeautifulSoup(html,"html.parser")
get_table_simple(content)

[['Month', 'Price'],
 ['July', '2', 'August4', 'September3', 'October2'],
 ['August', '4'],
 ['September', '3'],
 ['October', '2']]

#### With Tidying

In [20]:
!pip install pytidylib
from tidylib import tidy_document
from tidylib import tidy_fragment
html, errors = tidy_fragment(html)
html.replace("\n","")
content = BeautifulSoup(html,"html.parser")
get_table_simple(content)

OSError: Could not load libtidy using any of these names: libtidy,libtidy.so,libtidy-0.99.so.0,cygtidy-0-99-0,tidylib,libtidy.dylib,tidy

As you can see the tidying added a </tr> at the right place and we got our content parsed properly.

### Lib for fetching details

In [11]:

def get_ratios(url):
    page_response = requests.get(url, timeout=240)
    page_content, errors = tidy_document(page_response.content,options={'numeric-entities':1})
    page_content = BeautifulSoup(page_content, "html.parser")
    table_content = page_content.find_all('table',attrs={'class':'table4'})[-1]
    if "Data Not Available" in table_content.text:
         return {}
    
    table = get_table_simple(table_content)
    dates = table[2]
    rows = table[3:]
    ratios = {}
    ratios['dates'] = dates
    
    for row in rows:
        if len(row)>1:
            ratios[row[0]] = ffloat_list(row[1:])
    
    needed_keys = [('dates','ratios_dates'),
                   ('Diluted EPS (Rs.)','ratios_diluted_eps'),
                   ('Revenue from Operations/Share (Rs.)','ratios_revenue_per_share'),
                   ('PBT/Share (Rs.)','ratios_pbt_per_share'),
                   ('PBT Margin (%)','ratios_pbt_margin_per_share'),
                   ('Total Debt/Equity (X)','ratios_de'),
                   ('Asset Turnover Ratio (%)','ratios_asset_turnover_ratio'),
                   ('Current Ratio (X)','ratios_cr'),
                   ('EV/EBITDA (X)','ratios_ev_by_ebitda'),
                   ('Price/BV (X)','ratios_pb')]
    
    ratios = { your_key[1]: ratios[your_key[0]] if your_key[0] in ratios else [] for your_key in needed_keys }
    return ratios

In [12]:

def get_past_prices(sc_id):
    bse_url = "https://www.moneycontrol.com/tech_charts/bse/his/%s.csv"%sc_id
    nse_url = "https://www.moneycontrol.com/tech_charts/nse/his/%s.csv"%sc_id
    
    past_prices_nse = pd.read_csv(nse_url,header=None,names=['open','high','low','close','volume',1,2,3,4])[['open','high','low','close','volume']]
    past_prices_nse.index = pd.to_datetime(past_prices_nse.index)
    
    past_prices_bse = pd.read_csv(bse_url,header=None,names=['open','high','low','close','volume',1,2,3,4])[['open','high','low','close','volume']]
    past_prices_bse.index = pd.to_datetime(past_prices_bse.index)
    
    
    if len(past_prices_nse)>=len(past_prices_bse):
        past_prices = past_prices_nse
    else:
        past_prices = past_prices_bse
    return past_prices

In [13]:

def get_scrip_info(url,name):
    original_url = url
    key_val_pairs = {}
    key_val_pairs["original_url"] = original_url
    if not url.startswith("http"):
        url = "https://www.moneycontrol.com"+url
    try:
        page_response = requests.get(url, timeout=240)
        page_content = BeautifulSoup(page_response.content, "html.parser")
        scrip_name = name
        price = ffloat(page_content.find('div',attrs={'id':'Nse_Prc_tick_div'}).text.split(" ")[0].replace(',',''))
        name = page_content.find('h1',attrs={'class':'company_name'}).text
        
        yearly_high = page_content.find('span',attrs={'id':'n_52high'}).text.strip()
        yearly_low = page_content.find('span',attrs={'id':'n_52low'}).text.strip()
        html_data_content = page_content.find('div', attrs={'id': 'mktdet_1'})
        
        petable = get_table_simple(get_children(html_data_content)[0],is_table_tag=False)
        pbtable = get_table_simple(get_children(html_data_content)[1],is_table_tag=False)
        
        side_nav = page_content.find('dl',attrs={'id':'slider'})
        ratio_url = side_nav.find_all('dd')[2].find_all('a')[7]['href']
        ratio_url = "https://www.moneycontrol.com"+ratio_url
        ratios = get_ratios(ratio_url)
        
        
        volume = ffloat(page_content.find('span',attrs={'id':'nse_volume'}).text)
        
        sc_id = page_content.find('input',attrs={'id':'sc_id'}).get('value').lower()
        
        key_val_pairs = {**key_val_pairs, **ratios}
        
        past_prices = get_past_prices(sc_id)
        
        data_table = list()
        data_table.extend(petable)
        data_table.extend(pbtable)
        

        for row in data_table:
            k = row[0]
            if len(row)<2:
                v=None
            else:
                v = row[1].split(" ")[0].replace(',','')
            key_val_pairs[k]=v

        

        key_val_pairs["pe"] = ffloat(key_val_pairs.pop('P/E'))
        key_val_pairs["book_value"] = ffloat(key_val_pairs.pop('BOOK VALUE (Rs)'))
        key_val_pairs["deliverables"] = ffloat(key_val_pairs.pop('DELIVERABLES (%)'))
        key_val_pairs["eps"] = ffloat(key_val_pairs.pop('EPS (TTM)'))
        key_val_pairs["industry_pe"] = ffloat(key_val_pairs.pop('INDUSTRY P/E'))
        if 'MARKET CAP (Rs Cr)' in key_val_pairs:
            key_val_pairs["market_cap"] = key_val_pairs.pop('MARKET CAP (Rs Cr)')
        elif '**MARKET CAP (Rs Cr)' in key_val_pairs:
            key_val_pairs["market_cap"] = key_val_pairs.pop('**MARKET CAP (Rs Cr)')
        key_val_pairs["market_cap"] = ffloat(key_val_pairs["market_cap"])
        key_val_pairs["pb"] = ffloat(key_val_pairs.pop('PRICE/BOOK'))
        key_val_pairs["pc"] = ffloat(key_val_pairs.pop('P/C'))
        key_val_pairs['price'] = ffloat(price)
        key_val_pairs['volume'] = volume
        key_val_pairs["name"] = name
        key_val_pairs["scrip_name"] = scrip_name
        key_val_pairs["yearly_low"] = ffloat(yearly_low)
        key_val_pairs["yearly_high"] = ffloat(yearly_high)

        key_val_pairs['past_prices'] = past_prices
        key_val_pairs['de'] = np.nan
        key_val_pairs['ev_by_ebitda'] = np.nan
        if "ratios_ev_by_ebitda" in key_val_pairs and len(key_val_pairs["ratios_ev_by_ebitda"])>0:
            key_val_pairs['ev_by_ebitda'] = key_val_pairs["ratios_ev_by_ebitda"][0]
        
        if "ratios_de" in key_val_pairs and len(key_val_pairs["ratios_de"])>0:
            key_val_pairs['de'] = key_val_pairs["ratios_de"][0]
        key_val_pairs['failure'] = False

        del key_val_pairs['DIV (%)']
        del key_val_pairs['DIV YIELD.(%)']
        del key_val_pairs['FACE VALUE (Rs)']
        del key_val_pairs['Market Lot']
    except Exception as e:
        key_val_pairs['failure'] = True
        key_val_pairs['err'] = "Error for: %s"%original_url
        print(key_val_pairs['err'])
        return key_val_pairs
        
    return key_val_pairs


In [14]:
def get_scrip_info_by_nse_name(nse_name):
    url = "https://www.moneycontrol.com/mccode/common/autosuggesion.php?classic=true&query=%s&type=1&format=json"%nse_name
    page_response = requests.get(url, timeout=240)
    json_text = page_response.text
    data = json.loads(json_text)
    if len(data)>1:
        scrips = pd.DataFrame.from_records(data)["pdt_dis_nm"].values
        idx = list(map(lambda x:BeautifulSoup(x, "html.parser").find("span").text.split(",")[1].strip(),scrips)).index(nse_name)
        scrip_url = data[idx]['link_src']
    else:
        scrip_url = data[0]['link_src']
    return get_scrip_info(scrip_url,nse_name)

### Company Analysis and Filtering 



#### Generic Filter one variable upper lower limits

In [15]:
def get_generic_filter(param_name,lower_limit=None,upper_limit=None,
                                     replacement_nan=None,replacement_not_present=None):
    def filter_fn(stock_detail):
        param = replacement_not_present
        if param_name in stock_detail:
            param = ffloat(stock_detail[param_name])
            
        if np.isnan(param):
            param = replacement_nan
        
        if param is None or np.isnan(param):
            return False
        
        if param<=upper_limit and param>=lower_limit:
            return True
        return False
        
    return filter_fn

In [16]:
def get_all_company_details(nse_names,
                            threadpool_size=8):
    batch_size = 5*threadpool_size
    pool = ThreadPool(threadpool_size)
    scrip_details = pool.map(get_scrip_info_by_nse_name, nse_names)
    failures = list(filter(lambda x:x['failure'],scrip_details))
    successes = list(filter(lambda x:not x['failure'],scrip_details))
    successes={scrip['scrip_name']:scrip for scrip in successes}
    return successes,failures


In [21]:
some_nse_companies = ["AARTIDRUGS","ATUL","AUROPHARMA","BAJAJ-AUTO","BANKBARODA",
                     "DBCORP","DIVISLAB","EVEREADY","EICHERMOT",
                     "GLENMARK","GREAVESCOT","HDFCBANK","ITC","KOTAKBANK","KEC"]
successes,failures = get_all_company_details(some_nse_companies,
                        threadpool_size=4);

Error for: http://www.moneycontrol.com/india/stockpricequote/pharmaceuticals/aartidrugs/AD
Error for: http://www.moneycontrol.com/india/stockpricequote/chemicals/atul/A06
Error for: http://www.moneycontrol.com/india/stockpricequote/auto23wheelers/bajajauto/BA10
Error for: http://www.moneycontrol.com/india/stockpricequote/pharmaceuticals/aurobindopharma/AP
Error for: http://www.moneycontrol.com/india/stockpricequote/mediaentertainment/dbcorp/C13
Error for: http://www.moneycontrol.com/india/stockpricequote/bankspublicsector/bankofbaroda/BOB
Error for: http://www.moneycontrol.com/india/stockpricequote/drycells/evereadyindustriesindia/EII02
Error for: http://www.moneycontrol.com/india/stockpricequote/pharmaceuticals/divislaboratories/DL03
Error for: http://www.moneycontrol.com/india/stockpricequote/autolcvshcvs/eichermotors/EM
Error for: http://www.moneycontrol.com/india/stockpricequote/pharmaceuticals/glenmarkpharma/GP08
Error for: http://www.moneycontrol.com/india/stockpricequote/engines

In [18]:
len(successes.keys())
len(failures)

0

15

#### Write to File

In [ ]:

import dill
with open('tutorial_results.pkl', 'wb') as f:
    dill.dump(successes, f)
    dill.dump(failures, f)

#### Read from File 

In [ ]:
import dill
with open('tutorial_results.pkl', 'rb') as f:
    successes = dill.load(f)
    failures = dill.load(f)

### Portfolio Selection based on Data

In [ ]:
def filter_companies(all_scrips,filters=[]):
    scrip_details = list(all_scrips.values())
    
    for i in range(len(filters)):
        scrip_details = list(filter(filters[i],scrip_details))
    
    return scrip_details
    
    

In [ ]:
def get_df_from_scrip_details(scrip_details):
    other_cols = ['name','scrip_name']
    numeric_cols = ['price', 
                    'market_cap', 'pb', 'pe','de']
    
    
    all_cols = other_cols+numeric_cols
    scrip_details = [{ your_key: scrip[your_key] for your_key in all_cols } for scrip in scrip_details]
    scrip_details = pd.DataFrame.from_records(scrip_details)
    scrip_details[numeric_cols] = scrip_details[numeric_cols].applymap(ffloat)
    scrip_details = scrip_details[all_cols]
    return scrip_details

In [ ]:
pe_filter = get_generic_filter("pe",lower_limit=0,upper_limit=20,
                                     replacement_nan=-1,replacement_not_present=-1)
pb_filter = get_generic_filter("pb",lower_limit=0,upper_limit=4,
                                     replacement_nan=-1,replacement_not_present=-1)
filtered_companies = filter_companies(successes,filters=[pe_filter,pb_filter])
get_df_from_scrip_details(filtered_companies)

In [ ]:
de_filter = get_generic_filter("de",lower_limit=0,upper_limit=0.5,
                                     replacement_nan=0,replacement_not_present=0)
filtered_companies = filter_companies(successes,filters=[pe_filter,pb_filter,de_filter])
get_df_from_scrip_details(filtered_companies)

Below we see which filter passed and failed for different companies. This will help us understand why certain companies were rejected by our filtering process earlier.

In [ ]:
def score_company_on_filters(all_scrips,filters={}):
    all_scrips = list(all_scrips.values())
    other_cols = ['name','scrip_name']
    numeric_cols = ['price','market_cap', 'pb', 'pe','de']
    all_cols = other_cols+list(filters.keys())+numeric_cols
    scrip_details = []
    for scrip in all_scrips:
        for key in filters.keys():
            scrip[key] = filters[key](scrip)
        scrip_detail = { your_key: scrip[your_key] for your_key in all_cols }
        
        scrip_details.append(scrip_detail)
        
    scrip_details = pd.DataFrame.from_records(scrip_details)
    scrip_details[numeric_cols] = scrip_details[numeric_cols].applymap(ffloat)
    scrip_details = scrip_details[all_cols]
    return scrip_details
    

In [ ]:
scores_df = score_company_on_filters(successes,filters={"pe_filter":pe_filter,"pb_filter":pb_filter,"de_filter":de_filter})

In [ ]:
scores_df

### Plotting and Analysis

#### PE vs MarketCap Corr

In [ ]:
pe_list = []
mcap_list = []
for key in successes.keys():
    prices_df = successes[key]['past_prices']
    
    pe = successes[key]['pe']
    mcap = successes[key]['market_cap']
    
    pe_list.append(pe)
    mcap_list.append(mcap)
    
plt.figure(figsize=(12,6))
ax = sns.regplot(x=np.array(pe_list), y=np.array(mcap_list),fit_reg=True)
ax.set_xlim([10,40])
ax.set_ylim([0,6e5])
plt.xlabel("PE Ratio");
plt.ylabel("Market Cap in Crores (10 Million INR.)");

plt.show()
plt.figure(figsize=(12,6))
ax = sns.regplot(x=np.array(mcap_list), y=np.array(pe_list),fit_reg=True,logx=True)
plt.ylabel("PE Ratio");
plt.xlabel("Market Cap in Crores (10 Million INR.)");
plt.show()

#### Price Volume Charts

In [ ]:
def generate_price_chart(stock_df,name,days=1095,ewmas=[],other_technical_indicators=[]):
    plt.figure(figsize=(16,8))
    ts_df = stock_df.tail(days)
    handles = []
    p1, = plt.plot(ts_df.index, ts_df['close'],label="price")
    handles.append(p1)
    for ewma in ewmas:
        y = ts_df['close'].ewm(span=ewma).mean()
        p2, = plt.plot(ts_df.index, y,label="%s day ewma"%ewma)
        handles.append(p2)
    plt.legend(handles=handles)
    plt.title(name)
    plt.ylabel('Closing Price')
    plt.show()

In [ ]:

def generate_price_volume_chart(stock_df,name,days=1095,ewmas=[],other_technical_indicators=[]):
    plt.figure(figsize=(16,8))
    top = plt.subplot2grid((6,6), (0, 0), rowspan=4, colspan=6)
    bottom = plt.subplot2grid((6,6), (4,0), rowspan=2, colspan=6)
    ts_df = stock_df.tail(days)
    handles = []
    p1, = top.plot(ts_df.index, ts_df['close'],label="price")
    handles.append(p1)
    for ewma in ewmas:
        y = ts_df['close'].ewm(span=ewma).mean()
        p2, = top.plot(ts_df.index, y,label="%s day ewma"%ewma)
        handles.append(p2)
    top.legend(handles=handles)
    bottom.bar(ts_df.index, ts_df['volume']) 
    bottom.set_ylim([ts_df['volume'].min(),ts_df['volume'].max()])
    top.axes.get_xaxis().set_visible(False)
    top.set_title(name)
    top.set_ylabel('Closing Price')
    bottom.set_ylabel('Volume')
    
    plt.show()

In [ ]:
hero = get_scrip_info_by_nse_name('HEROMOTOCO')
exide = get_scrip_info_by_nse_name('EXIDEIND')

In [ ]:
generate_price_volume_chart(hero['past_prices'],"HERO Motocorp",days=252,ewmas=[7,45])

In [ ]:
generate_price_volume_chart(exide['past_prices'],"Exide",days=252,ewmas=[7,45])


#### Returns analysis

In [ ]:

def generate_returns_chart(stocks,days=252):
    plt.figure(figsize=(16,8))
    stocks = {key:stocks[key].tail(days).apply(lambda x: x / x[0]) for key in stocks.keys()}
    handles = []
    for key in stocks.keys():
        y = stocks[key]['close']
        p2, = plt.plot(stocks[key].index, y,label=key)
        handles.append(p2)
    plt.legend(handles=handles)
    plt.title("Comparative returns")
    plt.ylabel('Comparative Returns')
    
    plt.show()

In [ ]:
generate_returns_chart({"HERO":hero['past_prices'],
                       "EXIDE":exide['past_prices']})

#### Volatility analysis

In [ ]:
def generate_percent_change_chart(stocks,days=252):
    plt.figure(figsize=(16,8))
    stocks = {key:stocks[key].tail(days).pct_change()*100 for key in stocks.keys()}
    handles = []
    for key in stocks.keys():
        stocks[key]['name'] = key
        y = stocks[key]['close']
        p2, = plt.plot(stocks[key].index, y,label=key)
        handles.append(p2)
    all_stocks = pd.concat(list(stocks.values()))
    plt.legend(handles=handles)
    plt.title("Daily Percent Changes Chart")
    plt.ylabel('Daily Percent Changes')
    plt.show()
    fig, ax = plt.subplots(figsize=(16,8))
    sns.boxplot(x="name", y="close", data=all_stocks,ax=ax);
    ax.xaxis.set_tick_params(rotation=90)
    plt.show()

In [ ]:

generate_percent_change_chart({"HERO":hero['past_prices'],
                       "EXIDE":exide['past_prices']},days=252)

#### Returns and Volatility of all companies we used

In [ ]:
all_scrip_prices_df = {scrip["scrip_name"]:scrip['past_prices'] for scrip in successes.values()}


In [ ]:
generate_returns_chart(all_scrip_prices_df)
generate_percent_change_chart({scrip["scrip_name"]:scrip['past_prices'] for scrip in successes.values()})

#### Nifty 50, Nifty 100, Nifty midcap 50 Analysis

In [ ]:
n50_url = "http://www.moneycontrol.com/tech_charts/nse/his/nifty.csv"
n50 = pd.read_csv(n50_url,header=None,names=['open','high','low','close','volume'])[['open','high','low','close','volume']]
n50.index = pd.to_datetime(n50.index)


n100_url = "http://www.moneycontrol.com/tech_charts/nse/his/cnx_100.csv"
n100 = pd.read_csv(n100_url,header=None,names=['open','high','low','close','volume'])[['open','high','low','close','volume']]
n100.index = pd.to_datetime(n100.index)

nmid_url = "http://www.moneycontrol.com/tech_charts/nse/his/nifty_midcap_50.csv"
nmid = pd.read_csv(nmid_url,header=None,names=['open','high','low','close','volume'])[['open','high','low','close','volume']]
nmid.index = pd.to_datetime(nmid.index)




In [ ]:

generate_price_chart(n50,"nifty 50",days=1095,ewmas=[30,120])
generate_price_chart(n100,"nifty 100",days=1095,ewmas=[30,120])
generate_price_chart(nmid,"nifty midcap 50",days=1095,ewmas=[30,120])


In [ ]:
generate_returns_chart({"nifty 50":n50,"nifty 100":n100,"nifty midcap 50":nmid,"hero":hero['past_prices']['all_past_prices']})

In [ ]:



generate_percent_change_chart({"nifty 50":n50,"nifty 100":n100,"nifty midcap 50":nmid,"hero":hero['past_prices']['all_past_prices']},days=1095)


### References

- https://ntguardian.wordpress.com/2016/09/19/introduction-stock-market-data-python-1/
- https://mapattack.wordpress.com/2017/02/14/python-for-stocks-2/

more involved articles:

 - https://ntguardian.wordpress.com/2018/07/17/stock-data-analysis-python-v2/
 - https://nextjournal.com/hisham/stock-market
 